# App for testing how well our best model from stage 1 predicts


In machine learning you should always test your model with fresh data that the model has never seen before.

In this notebook, we do exactly that: we will create a new dataset for testing, and see how well the model does. In order to run this, the following assumptions should be fullfilled:

1. We have a saved model (which in fact is a pyspark.ml.PipelineModel) in Julio's HDFS home directory for the project (`"hdfs:///user/jsotovi2/pre2post/best_model_pre2post_yyyymmdd_hhmmss.sparkModel"`), where `yyyymmdd_hhmmss` is the datetime at which the model was saved. By default, this code will always grab the latest model in presence of more than one.
2. The following tables exist in Hive's metastore:
    + `raw_es.vf_pre_ac_final`
    + `raw_es.vf_pos_ac_final`
    + `raw_es.campaign_msisdncontacthist`
    + `raw_es.campaign_msisdnresponsehist`
    + `raw_es.anonymisation_lookup_msisdn` --> This is a temporary workaround. Chris knows about it.
3. Date and time clocks in the Spark driver are accurate. This is important because we rely heavilly on date in order to compute *which is the next month after the current one* and that sort of stuff.

With all that said, let's start:

## 1. The one and only line that we have to change between executions

To understand better the whole workflow:

1. We use historical data from one month (eg. 2017/04) to train a couple of models, and keep/save the best one (this is done in notebook #1, which you do not currently have). Once we have got a good model, we just save it to HDFS. This *does not has to happen every single month*; as long as the model is not extremely outated, there should be no need for running this all months.
2. We then use historical data from next month (2017/05) to get an unbiased measure on how good our saved model is. This is the notebook for this stage (notebook #2). This should run all months; it is always important to keep track of model performance on a monthly basis.
3. Finally, in order to make predictions, we will use the most recent data to predict customer behaviour. This can be ran as many times as we want, when needed (usually once a month).

Given that this notebook (#2) has to run on a monthly basis, I took care to structure the code so we just need to change one line from one monthly execution to another:

In [1]:
# The yyyymm date to verify people that have migrated
# from prepaid to postpaid.
# Should be manually changed:
month_for_verifying_in_pospago = "201707"

And that's it. All other date filters for all tables used are inferred from this one.

## 2. Imports and app setup

Your usual stuff:

In [2]:
# This literal_eval is needed since 
# we have to read from a textfile
# which is formatted as python objects.
# It is totally safe.
from ast import literal_eval

# Standard Library stuff:
from functools import partial
from datetime import date, timedelta, datetime

# Numpy stuff
from numpy import nan as np_nan, round as np_round

# Spark stuff
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.functions import (udf, col, decode, when, lit, lower, 
                                   translate, count, sum as sql_sum, max as sql_max, isnull)
from pyspark.sql.types import DoubleType, StringType, IntegerType

In [3]:
spark = (SparkSession.builder
         .appName("Pre2Post Spain hyperparameter tuning")
         .master("yarn")
         .config("spark.submit.deployMode", "client")
         .config("spark.ui.showConsoleProgress", "true")
         .enableHiveSupport()
         .getOrCreate()
         )

sc = spark.sparkContext

## 3. Data imports and first transformations

In [4]:
# This function will be very handy:
def get_next_month(dt):
    """
    Given a yyyymm string, returns the yyyymm string
    for the next month.
    """
    current_month = datetime.strptime(dt, "%Y%m")
    return (datetime(current_month.year, current_month.month, 28) + timedelta(days=4)).strftime("%Y%m")

Here we just use plain Python to compute a couple of dates derived from the one that we manually written. 

These dates (or rather, yyyymm strings) will be used as predicate for getting data from different Hive tables.

They are basically business rules.

In [5]:
# For the next 4 months
time_delta = (datetime.strptime(month_for_verifying_in_pospago, "%Y%m")
                                  - timedelta(5*365/12))

if time_delta.day > 16:
    if time_delta.month > 11:
        date_prepaid = datetime(time_delta.year + 1, 1, 1)
    else:
        date_prepaid = datetime(time_delta.year, time_delta.month + 1, 1)
else:
    date_prepaid = time_delta
    
month_for_getting_prepaid_data = date_prepaid.strftime("%Y%m")

# Do they migrate next month? (Well, really next 2 months due to delays in lists):
month_for_appearance_in_pospago = get_next_month(get_next_month(month_for_getting_prepaid_data))


print(month_for_getting_prepaid_data, month_for_appearance_in_pospago, month_for_verifying_in_pospago)

('201702', '201704', '201707')


Now, we will start to read tables. The first one is `raw_es.vf_pre_ac_final`, which contains information about *which VF clients were prepaid for a given month*, and very basic info about them (age, nationality, number of prepaid/postpaid services...)

The following cell includes reading the table and some transformations (that might can be easily translated to RedAgent):

In [6]:
# I know, it is atrocious, but the most straightforward/simple/robust
# way to select columns is with a simple, hardcoded Python list:

useful_columns_from_acFinalPrepago = ["FECHA_EJECUCION",
                                      "MSISDN",
                                      "NUM_DOCUMENTO_CLIENTE",
                                      "NACIONALIDAD",
                                      "NUM_PREPAGO",
                                      "NUM_POSPAGO",
                                      #"Tipo_Documento_Cliente", Very uninformed
                                      "Tipo_Documento_Comprador",
                                      "X_FECHA_NACIMIENTO"]

# Lots of tables in Hive have empty string instead
# of null for missing values in StringType columns:
def empty_str_to_null(string_value):
    """
    Turns empty strings to None, that are
    handled as nulls by Spark DataFrames:
    """
    if string_value == "":
        result = None
    elif string_value == u"":
        result = None
    else:
        result = string_value
    return result

# We register previous function as a udf:
empty_string_to_null = udf(empty_str_to_null, StringType())

# Function that returns customer age out of his/her birthdate:
def get_customer_age_raw(birthdate, month_for_getting_prepaid_data):
        if birthdate is None:
            return np_nan
        parsed_date = datetime.strptime(str(int(birthdate)), "%Y%m%d")
        timedelta = datetime.strptime(month_for_getting_prepaid_data, "%Y%m") - parsed_date
        return timedelta.days / 365.25

# We register previous function as a udf:
def get_customer_age_udf(birthdate, month):
    return udf(partial(get_customer_age_raw, month_for_getting_prepaid_data=month), DoubleType())(birthdate)


# Self-explanatory.
def subsitute_crappy_characters(string_column):
    """
    I really hate charset encoding.
    """
    return (string_column
            .replace(u"\ufffd", u"ñ")
            # add more here in the future if needed
           )

# We register previous function as a udf:
substitute_crappy_characters_udf = udf(subsitute_crappy_characters, StringType())



# And we finally read raw_es.vf_pre_ac_final,
# filtering by date, and with new columns
# that we create using our UDFs:
acFinalPrepago = (spark.read.table("raw_es.vf_pre_ac_final")
                  .where((col("year") == int(month_for_getting_prepaid_data[:4]))
                         & (col("month") == int(month_for_getting_prepaid_data[4:]))
                        )
                  #.select(*useful_columns_from_acFinalPrepago)
                  .withColumn("X_FECHA_NACIMIENTO", empty_string_to_null(col("X_FECHA_NACIMIENTO")))
                  .withColumn("NUM_DOCUMENTO_CLIENTE", empty_string_to_null(col("NUM_DOCUMENTO_CLIENTE")))
                  .withColumn("NUM_DOCUMENTO_COMPRADOR", empty_string_to_null(col("NUM_DOCUMENTO_COMPRADOR")))
                  .withColumn("age_in_years", get_customer_age_udf(col("X_FECHA_NACIMIENTO"),
                                                                   month_for_getting_prepaid_data)
                             )
                  .withColumn("NACIONALIDAD", substitute_crappy_characters_udf(col("NACIONALIDAD")))
                 )

# For some reason, pretty much all tables in Hive are
# REALLY underpartitioned (this is, with very few partitions
# given their size in number of rows). So, even though
# this is Spark's most common antipattern, I am going
# to actually repartition the DF to achieve some level
# of parallelism (and avoiding MemoryErrors in small executors):
acFinalPrepago = acFinalPrepago.repartition(int(acFinalPrepago.count() / 500)+1)

In [7]:
# In this acFinalPrepago DF we have a column (nationality)
# with lot's of different values (high cardinality), which
# is terrible for ML models, so we will get the most frequent
# countries, and replace all others with "Other":

most_frequent_countries = [u"España",
                           u"Marruecos",
                           u"Rumania",
                           u"Colombia",
                           u"Italia",
                           u"Ecuador",
                           u"Alemania",
                           u"Estados Unidos",
                           u"Francia",
                           u"Brasil",
                           u"Argentina",
                           u"Afganistan",
                           u"Bolivia",
                           u"Gran Bretaña",
                           u"Portugal",
                           u"Paraguay",
                           u"China",
                           u"Gran Bretana",
                           u"Venezuela",
                           u"Honduras",
                           u"Corea del Sur"]


acFinalPrepago = acFinalPrepago.withColumn("NACIONALIDAD", when(col("NACIONALIDAD").isin(most_frequent_countries),
                                                                  col("NACIONALIDAD")
                                                                 ).otherwise(lit("Other"))
                                            )

In [8]:
# Now we read another table: raw_es.vf_pos_ac_final,
# with some yyyymm predicate, and only two columns:
acFinalPospago_nextMonth = (spark.read.table("raw_es.vf_pos_ac_final")
                               .where((col("year") == int(month_for_appearance_in_pospago[:4]))
                                      & (col("month") == int(month_for_appearance_in_pospago[4:]))
                                     )
                               .select("x_id_red","x_num_ident")
                               .na.drop()
                               .withColumnRenamed("x_id_red", "x_id_red_NextMonth")
                               .withColumnRenamed("x_num_ident", "x_num_ident_NextMonth")
                              )

# Good old repartition for underpartitioned tables:
acFinalPospago_nextMonth = acFinalPospago_nextMonth.repartition(int(acFinalPospago_nextMonth.count() / 500)+1)

In [9]:
# And yet again, we read the same table, but with 
# different yyyymm predicate:
acFinalPospago_4monthsLater = (spark.read.table("raw_es.vf_pos_ac_final")
                               .where((col("year") == int(month_for_verifying_in_pospago[:4]))
                                      & (col("month") == int(month_for_verifying_in_pospago[4:]))
                                     )
                               .select("x_id_red","x_num_ident")
                               .na.drop()
                              )

# Good old repartition for underpartitioned tables:
acFinalPospago_4monthsLater = acFinalPospago_4monthsLater.repartition(int(acFinalPospago_4monthsLater.count() / 500)+1)

In [10]:
# And we perform one join:

join_prepago_pospago_1 = (acFinalPrepago
                         .join(acFinalPospago_nextMonth,
                               how="left",
                               on=(acFinalPrepago["MSISDN"]==acFinalPospago_nextMonth["x_id_red_NextMonth"])
                                & (acFinalPrepago["NUM_DOCUMENTO_COMPRADOR"]==acFinalPospago_nextMonth["x_num_ident_NextMonth"])
                             )
                       )

In [11]:
# And another:

join_prepago_pospago_2 = (join_prepago_pospago_1
                         .join(acFinalPospago_4monthsLater,
                              on=(join_prepago_pospago_1["x_id_red_NextMonth"]==acFinalPospago_4monthsLater["x_id_red"])
                                  & (join_prepago_pospago_1["x_num_ident_NextMonth"]==acFinalPospago_4monthsLater["x_num_ident"]),
                              how="left"
                             )
                        #.withColumn("migrated_to_postpaid", (~col("x_id_red").isNull()).cast(IntegerType()))
                       )

Up to this point, we have read 3 tables (well, tbh they were only 2, but one of them two times with different predicates), with some transformations, and we have joined them.

## 4. Contacts and responses table

We still need to join more information sources.

Next ones will be Telesales' contacts and responses, which also need some yyyymm predicates:

In [12]:
# Beautiful datetime manipulations:
datetime_for_appearance_in_pospago = datetime.strptime(month_for_appearance_in_pospago, "%Y%m")

datetime_min_contact = datetime((datetime_for_appearance_in_pospago - timedelta(days=8)).year,
                                (datetime_for_appearance_in_pospago - timedelta(days=8)).month,
                                1)

datetime_max_contact = datetime((datetime_for_appearance_in_pospago + timedelta(days=8)).year,
                                (datetime_for_appearance_in_pospago + timedelta(days=8)).month,
                                7)

month_for_getting_prepaid_data, month_for_appearance_in_pospago, datetime_min_contact, datetime_max_contact

('201702',
 '201704',
 datetime.datetime(2017, 3, 1, 0, 0),
 datetime.datetime(2017, 4, 7, 0, 0))

In [13]:
# Now, we read raw_es.campaign_msisdncontacthist
# with yyyymm predicates and other stupid business filters
# that I do not fully understand:
contacts = (spark.read.table("raw_es.campaign_msisdncontacthist")
            .where(col("contactdatetime") >= datetime_min_contact.strftime("%Y-%m-%d %H:%M:%S"))
            .where(col("contactdatetime") < datetime_max_contact.strftime("%Y-%m-%d %H:%M:%S"))
            #.where(col("contactdatetime") >= "2016-09-01 00:00:00")
            #.where(col("contactdatetime") < "2017-05-05 00:00:00")
            .where(col("CampaignCode").isin(['AUTOMMES_PXXXP_MIG_PROPENSOS']))
            .where(~(col("Canal").like("PO%")))
            .where(~(col("Canal").like("NBA%")))
            .where(col("Canal")=="TEL")
            .where(col("Flag_Borrado") == 0)
            )

# We read raw_es.campaign_msisdnresponsehist:
responses = (spark.read.table("raw_es.campaign_msisdnresponsehist")
            )

# We are going to join contacts DF with responses DF, and they
# happen to have columns with same names (but not same data),
# so we rename all columns in responses DF, adding responses_
# at the beggining:
responses_columns = [(column,"responses_"+column) for column in responses.columns]

for existing, new in responses_columns:
    responses = responses.withColumnRenamed(existing, new)

# Beautiful join. I do not expect you to understand
# it, because neither do I. I just translated some
# Teradata Query that VF Spain's CVM department uses
# to Spark DF syntax. It runs quite fast...
contacts_and_responses = (contacts.join(responses,
                                       how="left_outer",
                                       on=(contacts["TREATMENTCODE"]==responses["responses_TREATMENTCODE"])
                                          & (contacts["MSISDN"]==responses["responses_MSISDN"])
                                          & (contacts["CampaignCode"]==responses["responses_CampaignCode"])
                                       )
                                  .groupBy("MSISDN",
                                           "CAMPAIGNCODE",
                                           "CREATIVIDAD",
                                           "CELLCODE",
                                           "CANAL",
                                           "contactdatetime",
                                           "responses_responsedatetime")
                                  .agg(sql_max("responses_responsedatetime"))
                                  .select(col("MSISDN"), 
                                          col("CAMPAIGNCODE"), 
                                          col("CREATIVIDAD"), 
                                          col("CELLCODE"), 
                                          col("CANAL"), 
                                          col("contactdatetime").alias("DATEID"),
                                          when(isnull(col("max(responses_responsedatetime)")), "0")
                                              .otherwise("1").alias("EsRespondedor")
                                               
                                         )
                         ).withColumnRenamed("msisdn","msisdn_contact")

Cool. We have now only 2 DFs: one with *customer data* (the result of joining the first 3 tables), and this last one with contacts and responses.

We just have to join there two, and we will have our dataset almost prepared for ML...

## 4. Join of prepaid & postpaid with contacts and responses

But there is an issue. We have to join these 2 DFs on MSISDN, but it turns out that they have been anoymised differently, so we have to use an auxiliary lookup table inbetween...

In [14]:
# Here is the lookup table
lookup_msisdn = spark.read.table("raw_es.anonymisation_lookup_msisdn")

In [15]:
# Join between customer data and lookup table:
join_prepago_pospago_3 = join_prepago_pospago_2.join(lookup_msisdn,
                                                     how="left",
                                                     on=join_prepago_pospago_2["x_id_red"]==lookup_msisdn["cvm_value"]
                                                    )

In [16]:
# Another join, where we
# also create the target column for our 
# machine learning model:
join_prepago_pospago_4 = (join_prepago_pospago_3.join(contacts_and_responses,
                                                      how="left",
                                                      on=join_prepago_pospago_3["correct_value"]==contacts_and_responses["msisdn_contact"]
                                                     )
                                                .withColumn("migrated_to_postpaid", ((~col("msisdn_contact").isNull())
                                                                                     #&(~col("EsRespondedor").isNull())
                                                                                     ).cast(IntegerType()))
                        )

In [17]:
join_prepago_pospago_5 = (join_prepago_pospago_2
                          .where(col("x_id_red").isNull())
                          .join(lookup_msisdn,
                                how="left",
                                on=join_prepago_pospago_2["MSISDN"]==lookup_msisdn["cvm_value"]
                               )
                          )

join_prepago_pospago_6 = (join_prepago_pospago_5.join(contacts_and_responses,
                                                     how="left",
                                                     on=join_prepago_pospago_3["correct_value"]==contacts_and_responses["msisdn_contact"]
                                                     )
                          .withColumn("migrated_to_postpaid", lit("0").cast(IntegerType()))
                          .where(~(col("EsRespondedor").isNull()))
                          )    

In [18]:
join_prepago_pospago = (join_prepago_pospago_4
                        .where(col("migrated_to_postpaid")==1)
                        .union(join_prepago_pospago_6)
                        )

In [19]:
# Quick check that everything went fine.
# There should be lots of zeroes, and a 
# couple of ones (between 100 and 2000, depending
# on the month):
join_prepago_pospago.groupBy("migrated_to_postpaid").count().show()

+--------------------+-----+
|migrated_to_postpaid|count|
+--------------------+-----+
|                   1|  875|
|                   0|93899|
+--------------------+-----+



Finally. We have done pretty much all ETL needed. Everything so far is in `join_prepago_pospago`, which contains our dataset.

There is only one data source remaining, which is the one with pretty much all customer consumption patterns (MOU, MB, number of monthly calls...). We just have to read it, and join it with `join_prepago_pospago`:

In [20]:
# We will read raw_es.vf_pre_info_tarif.
# The columns that we care about are the following:

useful_columns_from_tarificadorPre = ['MSISDN',
                                      'MOU',
                                      'TOTAL_LLAMADAS',
                                      'TOTAL_SMS',
                                      'MOU_Week',
                                      'LLAM_Week',
                                      'SMS_Week',
                                      'MOU_Weekend',
                                      'LLAM_Weekend',
                                      'SMS_Weekend',
                                      'MOU_VF',
                                      'LLAM_VF',
                                      'SMS_VF',
                                      'MOU_Fijo',
                                      'LLAM_Fijo',
                                      'SMS_Fijo',
                                      'MOU_OOM',
                                      'LLAM_OOM',
                                      'SMS_OOM',
                                      'MOU_Internacional',
                                      'LLAM_Internacional',
                                      'SMS_Internacional',
                                      'ActualVolume',
                                      'Num_accesos',
                                      'Plan',
                                      'Num_Cambio_Planes',
                                      #'TOP_Internacional', # No idea of what is
                                      'LLAM_COMUNIDAD_SMART',
                                      'MOU_COMUNIDAD_SMART',
                                      'LLAM_SMS_COMUNIDAD_SMART',
                                      'Flag_Uso_Etnica',
                                      'cuota_SMART8',
                                      'cuota_SMART12',
                                      'cuota_SMART16']


# Read raw_es.vf_pre_info_tarif + yyyymm predicates + 
# column selection:
tarificadorPre = (spark.read.table("raw_es.vf_pre_info_tarif")
                  .where((col("year") == int(month_for_getting_prepaid_data[:4]))
                         & (col("month") == int(month_for_getting_prepaid_data[4:]))
                        )
                  .select(*useful_columns_from_tarificadorPre)
                 )

# Good old repartition for underpartitioned tables:
tarificadorPre = tarificadorPre.repartition(int(tarificadorPre.count() / 500)+1)


# Just as it happend with Nationlity column,
# Plan is a column with very high cardenality.
# We will replace any category not included
# in the following list with "Other":
plan_categories = ['PPIB7',
                   'PPFCL',
                   'PPIB4',
                   'PPXS8',
                   'PPIB8',
                   'PPIB9',
                   'PPTIN',
                   'PPIB1',
                   'PPVIS',
                   'PPREX',
                   'PPIB5',
                   'PPREU',
                   'PPRET',
                   'PPFCS',
                   'PPIB6',
                   'PPREY',
                   'PPVSP',
                   'PPIB2',
                   'PPIB3',
                   'PPRE2',
                   'PPRE5',
                   'PPVE2',
                   'PPVE1',
                   'PPRES',
                   'PPJ24',
                   'PPVE3',
                   'PPJAT',
                   'PPJMI']

tarificadorPre_2 = tarificadorPre.withColumn("Plan",
                                             when(col("Plan").isin(plan_categories),
                                                  col("Plan")
                                                 ).otherwise(lit("Other"))
                                            )

In [21]:
# Only one step left:
prepaid_dataset_1 = join_prepago_pospago.join(tarificadorPre_2,
                                               how="inner",
                                               on="MSISDN").persist(StorageLevel.DISK_ONLY_2)

`prepaid_dataset_1` is the DF that we will use for our model.

## 5. Feature engineering

Up until now, we have only integrated different sources, done some data cleansing, validation and preparation according to business rules. Now, we have to perform further preprocessing before feeding data to our machine learning model trained in notebook #1.

Let's start by removing some columns that, after a lot of local testing, we decided that are pretty much useless.

Also, we will separate numeric columns (IntegerType, DoubleType) from categorical columns (StringType), since it is a requirement to treat them differently before feeding them to any machine learning model:

In [22]:
numeric_columns = ['NUM_PREPAGO',
                   'NUM_POSPAGO',
                   'age_in_years',
                   #'documenttype_Other',
                   #'documenttype_cif',
                   #'documenttype_nif',
                   #'documenttype_pasaporte',
                   #'documenttype_tarj_residente',
                   #'nationality_Afganistan',
                   #'nationality_Alemania',
                   #'nationality_Argentina',
                   #'nationality_Bolivia',
                   #'nationality_Brasil',
                   #'nationality_China',
                   #'nationality_Colombia',
                   #'nationality_Corea_del_Sur',
                   #'nationality_Ecuador',
                   #'nationality_España',
                   #'nationality_Estados_Unidos',
                   #'nationality_Francia',
                   #'nationality_Gran_Bretana',
                   #'nationality_Gran_Bretaña',
                   #'nationality_Honduras',
                   #'nationality_Italia',
                   #'nationality_Marruecos',
                   #'nationality_Other',
                   #'nationality_Paraguay',
                   #'nationality_Portugal',
                   #'nationality_Rumania',
                   #'nationality_Venezuela',
                   #'migrated_to_postpaid',
                   'MOU',
                   'TOTAL_LLAMADAS',
                   'TOTAL_SMS',
                   'MOU_Week',
                   'LLAM_Week',
                   'SMS_Week',
                   'MOU_Weekend',
                   'LLAM_Weekend',
                   'SMS_Weekend',
                   'MOU_VF',
                   'LLAM_VF',
                   'SMS_VF',
                   'MOU_Fijo',
                   'LLAM_Fijo',
                   'SMS_Fijo',
                   'MOU_OOM',
                   'LLAM_OOM',
                   'SMS_OOM',
                   'MOU_Internacional',
                   'LLAM_Internacional',
                   'SMS_Internacional',
                   'ActualVolume',
                   'Num_accesos',
                   'Num_Cambio_Planes',
                   'LLAM_COMUNIDAD_SMART',
                   'MOU_COMUNIDAD_SMART',
                   'LLAM_SMS_COMUNIDAD_SMART',
                   #'Flag_Uso_Etnica',
                   'cuota_SMART8',
                   #'cuota_SMART12',
                   #'cuota_SMART16',
                   #'plan_PPFCL',
                   #'plan_PPFCS',
                   #'plan_PPIB1',
                   #'plan_PPIB2',
                   #'plan_PPIB3',
                   #'plan_PPIB4',
                   #'plan_PPIB5',
                   #'plan_PPIB6',
                   #'plan_PPIB7',
                   #'plan_PPIB8',
                   #'plan_PPIB9',
                   #'plan_PPJ24',
                   #'plan_PPJAT',
                   #'plan_PPJMI',
                   #'plan_PPRE2',
                   #'plan_PPRE5',
                   #'plan_PPRES',
                   #'plan_PPRET',
                   #'plan_PPREU',
                   #'plan_PPREX',
                   #'plan_PPREY',
                   #'plan_PPTIN',
                   #'plan_PPVE1',
                   #'plan_PPVE2',
                   #'plan_PPVE3',
                   #'plan_PPVIS',
                   #'plan_PPVSP',
                   #'plan_PPXS8'
                  ]

categorical_columns = ["tipo_documento_comprador", "NACIONALIDAD", "Plan"]

# We just rename our big DF...
prepaid_dataset_2 = prepaid_dataset_1

# In order to perform an easy, recursive
# typecasting:
for column in numeric_columns:
    prepaid_dataset_2 = prepaid_dataset_2.withColumn(column, col(column).cast(DoubleType()))
    

# Good old repartition for underpartitioned tables + 
# disk persistence:
prepaid_dataset_2 = (prepaid_dataset_2
                     .repartition(int(prepaid_dataset_2.count() / 50000)+1)
                     .persist(StorageLevel.DISK_ONLY_2)
                     )

## 6. Load machine learning model from HDFS

Now, we have to load our already trained (in notebook #1) model.

Machine model versioning is not easy. Just as any other software, it is a good practice to make snapshots of ML models.

Here, we will get the most recent model, basically reading from `"hdfs:///user/jsotovi2/pre2post/"`, listing all files, and getting the one with the latest `yyyymmmdd_hhmmss` in its name.

And given that hdfs3 (Python's API for reading from HDFS) is not installed and Snakebite is just a broken piece of software, we will just use this dirty hack to get the latest model name:

In [23]:
# Life sucks
import subprocess

directory_list = subprocess.check_output(["hadoop", "fs", "-ls", "/user/jsotovi2/pre2post_v2"]).split("\n")
files_list = [item.split(" ")[-1].split("/")[-1] for item in directory_list if "." in item.split(" ")[-1].split("/")[-1]]
history_list = ["_".join(theFile.replace(".txt","").split("_")[-2:]) 
                for theFile in files_list 
                if "model_pre2post_results" in theFile]

most_recent_model_date = list(reversed(sorted([datetime.strptime(a_date, "%Y%m%d_%H%M%S") for a_date in history_list])))[0]
most_recent_model_date_str = most_recent_model_date.strftime("%Y%m%d_%H%M%S")

In [24]:
# But at least we have the yyyymmdd_hhmmss string 
# of the latest model that exists on HDFS:
most_recent_model_date_str

'20170810_130214'

We have our model filename. We can just load it now:

In [25]:
from pyspark.ml import PipelineModel

most_recent_model = PipelineModel.load("hdfs:///user/jsotovi2/pre2post_v2/best_model_pre2post_"
                                       + most_recent_model_date_str + ".sparkModel")

## 7. Final data preparations

One column in our `prepaid_dataset_2` has a very uninformed column (with lots of missing values). We will fill it with the median computed in notebook #1, which happens to be saved as a TextFile in HDFS. Quite hacky, but it works:

In [46]:
# Get the median value for age:
training_results_file = sc.textFile("hdfs:///user/jsotovi2/pre2post_v2/best_model_pre2post_results_"
                               + most_recent_model_date_str + ".txt")

In [47]:
# We just read a textFile, which is a RDD of strings, formatted
# as Python objects. So if we use ast's literal_eval, everything
# will be fine:
training_results = dict([literal_eval(row) for row in training_results_file.collect()])

In [48]:
# And now we just have to fill the missing values with this median:
prepaid_dataset_2_filled = (prepaid_dataset_2
                            .na.fill(float(training_results["age_median_value"]), subset=["age_in_years"])
                            )

## 8. Model evaluation

Finally, our dataset is 100% prepared. Now we can feed it to the model:

In [49]:
predictions = most_recent_model.transform(prepaid_dataset_2_filled).cache()

Evaluate Area Under the ROC Curve in separate test month (Anything over 0.75 is fine):

In [50]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="probability", 
                                          labelCol="label", 
                                          metricName="areaUnderROC")

auc_test = evaluator.evaluate(predictions)

In [51]:
print(auc_test)

0.6564364994


## 9. Computing metrics

Some metrics cannot be computed with Spark (literally; due to some open bugs in sorting ordering), so we will export two columns of our dataset to Pandas DataFrame. In practice this is only ~6M rows x 2 columns, so it is not huge by any means and should not fill Spark Driver's memory:

In [59]:
results = predictions.select(udf(lambda x: x.tolist()[1], DoubleType())(col("probability")).alias("probability"),
                             col("prediction"),
                             col("label")).toPandas()

We're done with Spark. From this point to the end of this Notebook everything is pandas, scikit-learn and Bokeh. Feel free to skip this part.

In [60]:
import numpy as np
import pandas as pd

from bokeh.charts import Line, output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool

In [61]:
from sklearn.metrics import classification_report, confusion_matrix

classif_report = classification_report(results["label"], results["prediction"])

In [62]:
conf_matrix = pd.DataFrame(confusion_matrix(results["label"], results["prediction"]))

conf_matrix.index.name = "Real"
conf_matrix.columns.name = "Predicted"

In [63]:
def lift_curve(y_true, y_score, interval="decile", pos_label=None):
    """
    Compute the lift curve.

    Arguments:
        y_true: array, shape = [n_samples]
                True binary labels. If labels are not binary (0,1), 
                pos_label should be explicitly given

        y_score: array, shape = [n_samples, n_classes] or [n_samples]
                 Scores, can either be probability estimates (as created by
                 predict_proba()), confidence values, or non-thresholded measure 
                 of decisions (as returned by “decision_function” on some classifiers)
                 
                 If a (n_samples, n_classes) matrix is provided, pos_label
                 must be specified

        interval: since the lift curve is computed by some interval,
                  this allows the curve to be more or less granular.
                  Available options are "quartile" (4 points), "decile" (10 points)
                  and "percentile" (100 points)

        positiveClass:  an integer which indicates both the value of the
                        positive class in y_true, and the column position
                        in the matrix y_score which holds the probability for 
                        that positive class (None by default, but if y_score
                        has shape [n_samples, n_classes] you probably want
                        to set it to 1.0, since most binary classification 
                        problems assign the label 0 to negative class and 1 to 
                        positive class)

    Returns:
        A Python tuple with three arrays of shape (4,), (10,) or (100,) 
        (depending on interval argument). The first list are the percentiles, 
        the second one the lift values and the third one the score cutoffs 
        for each percentile
    """
    if interval == "decile":
        n_points = 10
    elif interval == "quartile":
        n_points = 4
    elif interval == "percentile":
        n_points = 100
    else:
        raise NotImplementedError('Available intervals are "percentile","decile" and "quartile".')
    
    if not isinstance(y_score, np.ndarray):
        y_score = np.array(y_score)
        
    # A (n_samples, n_classes) array is provided:
    if len(y_score.shape) > 1:
        if pos_label is None:
            raise ValueError("Since y_score is a matrix, you must provide pos_label.")
        pos_label = int(pos_label)
        sorting_index = np.argsort(-y_score[:,pos_label])
        sorted_probas = y_score[:,pos_label][sorting_index]
    else:
        sorting_index = np.argsort(-y_score)
        sorted_probas = y_score[sorting_index]
    
    sorted_y_true = y_true[sorting_index]
    
    classes_and_counts = list(enumerate(np.bincount(sorted_y_true.astype(np.int))))
    num_observations = float(len(sorted_y_true))
    
    percentage_positive_class = ([count for class_, count in classes_and_counts if class_==pos_label][0]
                                 / num_observations
                                )
        
    observations_per_point = int(num_observations / float(n_points)) + 1
    
    lift_points = []
    percentile_scores = []
    
    for point in range(1, n_points+1):
        
        chosen_interval = sorted_y_true[:observations_per_point * point]
        interval_classes_and_counts = list(enumerate(np.bincount(chosen_interval.astype(np.int))))
        
        try:
            percentage_positive_class_in_interval = ([count for class_, count in interval_classes_and_counts if class_==pos_label][0]
                                                     / float(len(chosen_interval))
                                                    )
            
        except IndexError: # If no positive observation in this interval...
            percentage_positive_class_in_interval = 0.0
            
        lift_in_interval = percentage_positive_class_in_interval / float(percentage_positive_class)
        lift_points.append(lift_in_interval)
        percentile_scores.append(sorted_probas[:observations_per_point * point][-1])

    lift_coordinates = list(zip([num / float(len(range(1, n_points+1))) for num in range(1, n_points+1)], 
                                lift_points, 
                                percentile_scores)
                           )
    
    return (np.array([i[0] for i in lift_coordinates]), 
            np.array([i[1] for i in lift_coordinates]),
            np.array([i[2] for i in lift_coordinates])
           )

In [64]:
percentiles, lifts, scores = lift_curve(y_true=results["label"],
                                        y_score=results["probability"],
                                        interval="percentile",
                                        pos_label=1.0)

lift_df = pd.DataFrame({"percentile_number":percentiles,
                        "lift":lifts,
                        "score":scores})

lift_df["percentile"] = np_round(lift_df["percentile_number"], 4).astype(str)
lift_df["cutoff_score"] = np_round(lift_df["score"], 6).astype(str)

In [65]:
lift_source = ColumnDataSource(data=lift_df)

hover = HoverTool(tooltips=[("Cutoff score", "@cutoff_score"),
                            ("Percentile", "@percentile"),
                            ("Lift value", "@lift")]
                 )

lift_plot = figure(plot_width=700, plot_height=480, tools=["pan",
                                                           "box_zoom",
                                                           "wheel_zoom",
                                                           hover,
                                                           "save",
                                                           "reset"],
                   x_axis_label="Percentile",
                   y_axis_label="Lift",
                   x_range=(lift_df["percentile_number"].min(), lift_df["percentile_number"].max()), 
                   y_range=(lift_df["lift"].min(), lift_df["lift"].max()),
                   title="Lift Curve")

# Lift Curve
lift_plot.line("percentile_number", 
               "lift", 
               line_width=3, 
               color="#ff0000",
               source=lift_source)

# Color in Area Under Lift
lift_plot.patch(lift_df["percentile"].tolist() + [lift_df["percentile_number"].min()],
                lift_df["lift"].tolist() + [lift_df["lift"].min()],
                fill_alpha=0.1,
                fill_color="#ff6666",
                line_alpha=0.0
               )

GlyphRenderer(id='058df438-a6b6-4546-810c-010402d02575', ...)

In [66]:
# ROC calculation
from sklearn.metrics import roc_curve, roc_auc_score


fpr, tpr, scores = roc_curve(y_true=results["label"],
                             y_score=results["probability"],
                             pos_label=1.0
                            )
df_roc_test = pd.DataFrame({"fpr":fpr, 
                            "tpr":tpr,
                            "score": scores})

df_roc_test["fpr_nice"] = np_round(df_roc_test["fpr"], 6).astype(str)
df_roc_test["tpr_nice"] = np_round(df_roc_test["tpr"], 6).astype(str)
df_roc_test["cutoff_score"] = np_round(df_roc_test["score"], 6).astype(str)

auc_test = roc_auc_score(results["label"],
                         results["probability"],
                         average=None)

In [67]:
# ROC plotting
roc_source = ColumnDataSource(data=df_roc_test)

hover_roc = HoverTool(names=["roc"],
                      tooltips=[("Cutoff score", "@cutoff_score"),
                                ("True Positive Rate", "@tpr_nice"),
                                ("False Positive Rate", "@fpr_nice"),]
                     )

roc_plot = figure(plot_width=700, plot_height=480, tools=["pan",
                                                          "box_zoom",
                                                          "wheel_zoom",
                                                          hover_roc,
                                                          "save",
                                                          "reset"],
                  x_axis_label="False Positive Rate",
                  y_axis_label="True Positive Rate",
                  title="ROC Curve")

# ROC curve
roc_plot.line("fpr", 
              "tpr", 
              line_width=3, 
              color="#ff0000",
              name="roc",
              source=roc_source)

# Random guess
roc_plot.line([0.0, 1.0],
              [0.0, 1.0],
              line_width=2,
              color="#FA58AC",
              line_dash='dashed',
              name="random_guess")

# Color Area under ROC
roc_plot.patch(df_roc_test["fpr"].tolist() + [1.0],
               df_roc_test["tpr"].tolist() + [0.0],
               fill_alpha=0.1,
               fill_color="#ff6666",
               line_alpha=0.0,
               legend="AUC: %.4f" % auc_test
              )

roc_plot.legend.location = "bottom_right"

## 10. Results

In [68]:
output_notebook()

Loading BokehJS ...

In [69]:
print(classif_report)

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     93899
        1.0       0.00      0.00      0.00       875

avg / total       0.98      0.99      0.99     94774



In [70]:
conf_matrix

Predicted,0,1
Real,,
0,93899,0
1,875,0


In [71]:
show(lift_plot)

In [72]:
show(roc_plot)

# TODO:

+ Figure out a way to export these charts and results to... HDFS? 